In [ ]:
import torch
from torch.utils.data import DataLoader


from data_mgmt import RecSysData as rsd
from core.loops import train_loop, test_loop
from data_mgmt import ValidationBaseDataClass
from data_mgmt.RecSysDataTest import TestRecSysData
from models.FactorioMachine import FactorizationMachineModel

# default imports because meh
import numpy as np
import pandas as pd
import seaborn as sns
import os
from copy import deepcopy

In [ ]:
# # # # # # # # # # # # # # # #
# INPUTS TO POST PROCESSING!  #
# # # # # # # # # # # # # # # #
ppath   =   os.path.abspath('../data/')
MODEL   =   "adam_256_001_1e4_500_800_FULL"+".pkl"
PATH    =   os.path.join(os.path.abspath('models/saved/'),MODEL)
OUTPATH =   os.path.abspath("../output/")

test_data_file      = os.path.join(ppath,"pairs_Rating.txt")
RESULTS_FILE_NAME   = os.path.join(OUTPATH,"yeah_boye3.txt")

In [ ]:
ds_train = rsd.RecSysData(ppath)

ds_test  = TestRecSysData(ds_train.df_data,
                        test_data_file,
                        transform=ds_train.transform)

tdl = DataLoader(ds_train, batch_size=128, shuffle=False)
tst = DataLoader(ds_test, batch_size=128, shuffle=False)

n_user = ds_train.df_data.uid.append(ds_test.df_data.uid).unique().shape[0]
n_item = ds_train.df_data.pid.append(ds_test.df_data.pid).unique().shape[0]
model_dims = np.array([n_user,n_item])

model = FactorizationMachineModel(model_dims, 512+128)
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
model

In [ ]:
def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss = torch.zeros(1)
    predictions = torch.Tensor([])

    with torch.no_grad():
        for X in dataloader:
            pred = model(X.type(torch.long)).flatten()
            predictions = torch.cat((predictions,pred),axis=-1)

    return predictions

In [ ]:
PREDICTIT = test_loop(tst, model)
outoutout=ds_test.df_data.copy()
outoutout['rating']=PREDICTIT.numpy()
ininin = ds_train.df_data.copy()

outoutout.loc[outoutout.rating < 1.0,'rating'] = 1.0
outoutout.loc[outoutout.rating > 5.0,'rating'] = 5.0

ininin['set']='train'
outoutout['set']='test'
full=ininin.append(outoutout)

In [ ]:
sns.displot(data=full.reset_index(),x='rating',kind='ecdf',hue='set')

In [ ]:
with open(RESULTS_FILE_NAME,'w') as filee:
    filee.write("reviewerID-itemID,prediction\n")
    for i in range(len(outoutout)):
        filee.write(f"{outoutout.reviewerID.iloc[i]}-{outoutout.itemID.iloc[i]},{outoutout.rating.iloc[i]}\n")